<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=62dce0ca0e08f53b52e5a12b536ab72a7adc8abb79e284e0d20e83c39aae1296
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-23 13:47:19
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.42 C
-------------------
Today PnL: 25.72 K (0.18%)
Current PnL: -25.46 L (-16.45%)
CY Booked + Current PnL: -11.17 L (-7.21%)
-------------------
Total profit:  1.62 L
Total loss:  -27.08 L
-------------------
Total Booked + Current PnL: 15.81 L (12.37%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.09%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 90.09 L (63.63%)
Deployed:  1.28 C
Current:  1.42 C
CAGR/XIRR %: 5.65%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-1.29,-19.07,23.68,0.09,19312.0,-19223.0,81554.0,89.33,39.0,M-SC,2.39,253.0,-1.00,0.58,7.14,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,-0.22,8.35,10.47,19.69,21751.0,16008.0,207750.0,-2.18,54.0,X-MC,2.48,78.0,0.74,1.47,15.71,XY25,NTT,AC
50,MASFIN,398.61,1.33,-1.97,24.51,22.05,23541.0,-1935.0,96045.0,-15.54,55.0,H-SC,6.35,164.0,-0.08,0.68,39.61,XR,ATH,FINANCE
43,ITC,452.00,0.56,-0.75,11.62,10.78,27716.0,-1802.0,238516.0,-40.74,54.0,X-LC,2.11,5.0,-0.07,1.69,4.45,X40,NTT,FMCG
52,NESTLEIND,1377.00,-0.33,10.46,9.90,21.40,29027.0,27776.0,293202.0,2.52,54.0,X-LC,5.47,12.0,0.96,2.08,17.80,XY25,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TITAGARH,1548.0,3.68,-20.30,82.86,45.74,159931.0,-49164.0,193013.0,6.29,60.0,H-SC,6.29,174.0,-0.31,1.37,25.54,XY24,NTT,ENGINEERING
69,SURYODAY,216.0,3.58,-20.82,55.40,23.03,78546.0,-37291.0,141780.0,53.51,53.0,H-SC,7.62,167.0,-0.47,1.01,40.67,XR,NTT,BANKS
58,RECLTD,446.0,3.37,-3.65,25.32,20.74,49563.0,-7425.0,195745.0,26.46,56.0,M-MC,8.47,189.0,-0.15,1.39,6.57,XY25,NTT,FINANCE
18,COFFEEDAY,80.0,3.32,-39.43,123.34,35.27,84826.0,-44775.0,68774.0,-53.72,51.0,L-SC,9.73,270.0,-0.53,0.49,68.33,XR,NTT,HOTELS
79,VAIBHAVGBL,521.0,3.14,-25.24,115.05,60.77,157204.0,-46135.0,136640.0,57.58,53.0,H-SC,4.71,159.0,-0.29,0.97,23.05,XR,NTT,JEWELLERY


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJESHEXPO,518.00,-4.95,-57.65,136.26,0.07,79168.0,-79076.0,58101.0,2030.05,67.0,L-SC,3.57,268.0,-1.00,0.41,44.50,OX40N,NTT,JEWELLERY
6,ASIANTILES,137.00,-2.96,-5.08,91.39,81.67,81379.0,-4764.0,89046.0,8016.67,65.0,L-SC,12.10,271.0,-0.06,0.63,72.44,XR,NTT,CERAMICS
14,CAMPUS,393.00,-2.83,-12.67,52.62,33.28,77098.0,-21257.0,146518.0,-29.66,38.0,M-SC,1.89,222.0,-0.28,1.04,13.56,XY24,NTT,FOOTWEAR
83,WHIRLPOOL,2270.00,-1.67,-24.54,141.08,81.92,122213.0,-28171.0,86627.0,-55.82,32.0,M-SC,20.66,236.0,-0.23,0.61,3.34,XR,NTT,DURABLES
25,FINCABLES,1641.55,-1.45,-2.07,112.31,107.91,162387.0,-3056.0,144588.0,-20.15,54.0,M-SC,11.93,216.0,-0.02,1.03,7.81,OX40N,ATH,CABLES


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-1.45,-2.07,112.31,107.91,162387.0,-3056.0,144588.0,-20.15,54.0,M-SC,11.93,216.0,-0.02,1.03,7.81,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.69,-16.75,20.14,0.02,29269.0,-29231.0,145328.0,121.54,41.0,M-SC,12.03,234.0,-1.00,1.03,26.41,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-1.29,-19.07,23.68,0.09,19312.0,-19223.0,81554.0,89.33,39.0,M-SC,2.39,253.0,-1.00,0.58,7.14,OX40N,NTT,DURABLES
72,TATAELXSI,9161.00,-0.49,-21.71,68.65,32.03,70852.0,-28627.0,103208.0,-15.20,62.0,H-SC,9.10,148.0,-0.40,0.73,15.60,OX40N,NTT,IT
66,SIS,528.00,-0.72,-24.05,59.66,21.26,50310.0,-26704.0,84328.0,1989.64,50.0,H-SC,4.73,166.0,-0.53,0.60,13.94,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,-0.38,11.41,54.62,72.27,103117.0,19342.0,188790.0,-4.26,75.0,M-LC,3.68,99.0,0.19,1.34,18.29,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-0.38,11.41,54.62,72.27,103117.0,19342.0,188790.0,-4.26,75.0,M-LC,3.68,99.0,0.19,1.34,18.29,XR,NTT,IT
25,FINCABLES,1641.55,-1.45,-2.07,112.31,107.91,162387.0,-3056.0,144588.0,-20.15,54.0,M-SC,11.93,216.0,-0.02,1.03,7.81,OX40N,ATH,CABLES
38,INDIAMART,4810.62,1.49,-3.26,113.69,106.72,135648.0,-4022.0,119314.0,-52.15,43.0,H-SC,4.26,139.0,-0.03,0.85,19.35,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,0.88,-2.32,38.85,35.63,79531.0,-4862.0,204712.0,-16.30,46.0,H-MC,2.39,119.0,-0.06,1.45,13.97,AR,ATH,PHARMA
6,ASIANTILES,137.00,-2.96,-5.08,91.39,81.67,81379.0,-4764.0,89046.0,8016.67,65.0,L-SC,12.10,271.0,-0.06,0.63,72.44,XR,NTT,CERAMICS


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,0.02,-17.83,89.22,55.47,178348.0,-43389.0,199897.0,-27.18,32.0,X-MC,15.19,64.0,-0.24,1.42,2.42,X40N,ATH,FINANCE
34,HONAUT,58357.33,-0.68,-18.05,74.88,43.32,99950.0,-29392.0,133480.0,-29.54,32.0,X-SC,5.65,90.0,-0.29,0.95,2.49,X40N,ATH,ELECTRICAL
22,DIXON,18931.72,-0.13,-14.58,47.58,26.07,79346.0,-28457.0,166764.0,-58.46,33.0,X-MC,11.86,56.0,-0.36,1.18,3.86,X40N,ATH,IT
8,AWL,485.00,-0.74,-24.08,102.17,53.50,234076.0,-72647.0,229104.0,-63.25,33.0,X-MC,5.40,58.0,-0.31,1.63,1.76,XY24,NTT,FMCG
20,DABUR,735.00,-0.23,-3.70,49.22,43.70,117580.0,-9181.0,238887.0,-11.62,35.0,X-MC,2.08,72.0,-0.08,1.70,10.79,XY24,BTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BERGEPAINT,680.0,0.62,-2.82,24.66,21.14,54481.0,-6411.0,220928.0,-9.65,47.0,X-MC,1.18,74.0,-0.12,1.57,24.53,XY24,NTT,PAINTS
15,CAMS,950.0,-0.21,-0.34,24.90,24.48,62499.0,-858.0,250998.0,-81.09,49.0,X-SC,1.18,86.0,-0.01,1.78,22.72,X40N,NTT,MISC
20,DABUR,735.0,-0.23,-3.70,49.22,43.70,117580.0,-9181.0,238887.0,-11.62,35.0,X-MC,2.08,72.0,-0.08,1.70,10.79,XY24,BTT,FMCG
43,ITC,452.0,0.56,-0.75,11.62,10.78,27716.0,-1802.0,238516.0,-40.74,54.0,X-LC,2.11,5.0,-0.07,1.69,4.45,X40,NTT,FMCG
78,UNITDSPR,1644.0,0.25,5.06,14.95,20.77,36776.0,11850.0,245994.0,-4.14,51.0,X-MC,2.11,62.0,0.32,1.75,12.33,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-1.37,-26.12,122.18,64.15,214792.0,-62151.0,175800.0,-56.75,37.0,X-SC,3.82,82.0,-0.29,1.25,0.37,XY24,BTT,CEMENT
19,COLPAL,3726.84,-0.25,-20.15,77.22,41.51,162394.0,-53065.0,210300.0,-4.98,39.0,X-MC,7.74,61.0,-0.33,1.49,0.75,XY25,ATH,FMCG
11,BATAINDIA,2096.00,0.11,-38.78,120.85,35.20,95195.0,-49899.0,78771.0,3.25,35.0,X-SC,16.87,92.0,-0.52,0.56,1.04,X40,NTT,FOOTWEAR
59,RELAXO,1176.00,0.24,-47.76,191.56,52.31,145260.0,-69330.0,75830.0,-44.34,43.0,X-SC,7.26,91.0,-0.48,0.54,1.50,X40N,NTT,FOOTWEAR
8,AWL,485.00,-0.74,-24.08,102.17,53.50,234076.0,-72647.0,229104.0,-63.25,33.0,X-MC,5.40,58.0,-0.31,1.63,1.76,XY24,NTT,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,0.26,-28.31,99.24,42.84,46251.0,-18401.0,46605.0,-53.17,50.0,X-SC,34.21,83.0,-0.40,0.33,5.44,XY24,NTT,MISC
59,RELAXO,1176.00,0.24,-47.76,191.56,52.31,145260.0,-69330.0,75830.0,-44.34,43.0,X-SC,7.26,91.0,-0.48,0.54,1.50,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.11,-38.78,120.85,35.20,95195.0,-49899.0,78771.0,3.25,35.0,X-SC,16.87,92.0,-0.52,0.56,1.04,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,1.52,-0.97,25.30,24.08,31806.0,-1234.0,125716.0,-9.02,48.0,X-SC,6.12,89.0,-0.04,0.89,18.45,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.68,-18.05,74.88,43.32,99950.0,-29392.0,133480.0,-29.54,32.0,X-SC,5.65,90.0,-0.29,0.95,2.49,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-0.69,-9.21,32.95,20.71,113153.0,-34829.0,343408.0,-21.68,68.0,X-LC,3.47,1.0,-0.31,2.44,14.75,X40,ATH,IT
41,INFY,1972.00,-1.31,13.52,18.26,34.24,64856.0,42290.0,355178.0,-11.70,68.0,X-LC,5.81,2.0,0.65,2.52,22.92,X40,NTT,IT
75,TMPV,600.00,1.36,-25.38,64.81,22.98,109240.0,-57343.0,168555.0,-23.08,55.0,X-LC,6.40,3.0,-0.52,1.20,6.01,XY24,NTT,AUTO
81,VBL,671.64,-0.84,-2.73,39.47,35.67,121263.0,-8613.0,307229.0,-14.36,57.0,X-LC,3.61,4.0,-0.07,2.18,10.75,X40N,ATH,FMCG
43,ITC,452.00,0.56,-0.75,11.62,10.78,27716.0,-1802.0,238516.0,-40.74,54.0,X-LC,2.11,5.0,-0.07,1.69,4.45,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-0.89,-37.34,112.08,32.89,54223.0,-28828.0,48379.0,-697.80,54.0,L-MC,6.80,259.0,-0.53,0.34,33.23,XR,NTT,BANKS
56,RAJESHEXPO,518.00,-4.95,-57.65,136.26,0.07,79168.0,-79076.0,58101.0,2030.05,67.0,L-SC,3.57,268.0,-1.00,0.41,44.50,OX40N,NTT,JEWELLERY
6,ASIANTILES,137.00,-2.96,-5.08,91.39,81.67,81379.0,-4764.0,89046.0,8016.67,65.0,L-SC,12.10,271.0,-0.06,0.63,72.44,XR,NTT,CERAMICS
50,MASFIN,398.61,1.33,-1.97,24.51,22.05,23541.0,-1935.0,96045.0,-15.54,55.0,H-SC,6.35,164.0,-0.08,0.68,39.61,XR,ATH,FINANCE
13,BSOFT,831.70,1.28,-17.35,82.43,50.79,95828.0,-24399.0,116254.0,5.91,74.0,H-SC,7.64,151.0,-0.25,0.82,35.36,XR,ATH,IT


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,1.28,-17.35,82.43,50.79,95828.0,-24399.0,116254.0,5.91,74.0,H-SC,7.64,151.0,-0.25,0.82,35.36,XR,ATH,IT
56,RAJESHEXPO,518.00,-4.95,-57.65,136.26,0.07,79168.0,-79076.0,58101.0,2030.05,67.0,L-SC,3.57,268.0,-1.00,0.41,44.50,OX40N,NTT,JEWELLERY
61,ROUTE,2227.21,-0.03,-45.65,211.04,69.05,152626.0,-60745.0,72321.0,-56.18,55.0,H-SC,20.98,140.0,-0.40,0.51,11.53,SR,ATH,IT
84,WIPRO,420.00,-0.38,11.41,54.62,72.27,103117.0,19342.0,188790.0,-4.26,75.0,M-LC,3.68,99.0,0.19,1.34,18.29,XR,NTT,IT
6,ASIANTILES,137.00,-2.96,-5.08,91.39,81.67,81379.0,-4764.0,89046.0,8016.67,65.0,L-SC,12.10,271.0,-0.06,0.63,72.44,XR,NTT,CERAMICS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.37
1,25,44.83
2,50,76.55


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.97
MC    30.79
LC    24.26
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.81
X40      23.05
X40N     12.96
XR        9.94
XY25      9.06
AR        8.92
OX40N     7.53
SR        1.00
MH        0.75
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.26
H-SC    23.87
X-LC    20.80
M-SC    11.57
X-SC     8.00
H-MC     4.80
L-SC     1.53
M-MC     1.39
M-LC     1.34
H-LC     1.11
L-LC     1.01
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.19,-7.68,42.30
IT,13.16,-13.24,70.17
FINANCE,9.57,-18.24,69.67
MISC,7.28,-26.67,78.48
ELECTRICAL,5.96,-11.74,52.67
PAINTS,5.78,-13.25,30.07
INSURANCE,4.75,-1.37,36.06
PHARMA,4.09,-2.10,34.24
AUTO,2.87,-29.80,74.80


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3147890.0,21
AR,1313795.0,10
XR,1260337.0,13
X40,975835.0,14
X40N,860577.0,9
OX40N,730972.0,10
XY25,362497.0,6
SR,278769.0,2
MH,78675.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3615242.0,25
M-SC,1429421.0,15
X-MC,1411067.0,16
X-LC,793496.0,11
X-SC,792292.0,8
H-MC,402259.0,3
L-SC,245373.0,3
M-LC,103117.0,1
H-LC,76823.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1260079.0      6
           AR         902015.0      5
M-SC       XY24       812206.0      6
H-SC       XR         763425.0      7
X-MC       X40        483245.0      7
           XY24       406137.0      3
           X40N       337540.0      4
H-SC       OX40N      332279.0      4
M-SC       OX40N      319525.0      5
X-SC       X40N       307709.0      3
X-LC       X40        300856.0      5
X-SC       XY24       292849.0      3
H-SC       SR         278769.0      2
X-LC       X40N       215328.0      2
H-MC       AR         210634.0      2
X-SC       X40        191734.0      2
H-MC       XY24       191625.0      1
X-LC       XY24       184994.0      2
X-MC       XY25       184145.0      2
M-SC       XR         173367.0      2
L-SC       XR         166205.0      2
M-SC       AR         124323.0      2
M-LC       XR         103117.0      1
X-LC       XY25        92318.0      2
L-SC       OX40N       79168.0      1
H-SC       MH          78675.0      1
H-LC       AR          76823.0      1
L-MC       XR          54223.0      1
M-MC       XY25        49563.0      1
L-LC       XY25        36471.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 51.0 seconds
